In [1]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 自身のColab環境によってディレクトリを変更して下さい
import os
os.chdir("/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/")
!pip install transformers
!pip install colorama
!pip install sentencepiece
!pip install mojimoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#!/usr/bin/env python
# coding: utf-8

# ### 参考notebook
# https://www.kaggle.com/code/debarshichanda/pytorch-w-b-jigsaw-starter

# In[1]:


# !nvidia-smi


# In[2]:


# !pip install --upgrade wandb


# In[3]:


# !pip install GPUtil
# !pip install transformers==4.12.2
# !pip install fugashi
# !pip install mecab-python3
# !pip install ipadic

# !pip install colorama


# In[4]:


import os
import gc
import copy
import time
import random
import string
import sys

import datetime
from datetime import datetime, timedelta, timezone

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoConfig, AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import re
import unicodedata

import regex
import scipy as sp
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, AdamW, T5Tokenizer
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

sys.path.append("/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/roberta_japanese")
from tokenization_roberta_japanese import RobertaJapaneseTokenizer

warnings.filterwarnings('ignore')


# In[5]:


# import wandb
# wandb.login()


# In[6]:


def id_generator(size=12, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))

JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
current_time = now.strftime("%Y-%m-%d-%H-%M", )

HASH_NAME = "Fakenews-" + current_time

print(HASH_NAME)


# In[7]:


# from google.colab import drive 
# drive.mount('/content/drive')


# In[8]:


ROOT = "../../cpt-fakenews"
# CUR_DIR = ROOT+"/baseline/code"
# %cd $CUR_DIR


# In[9]:


CONFIG = {"seed": 2022,
          "epochs": 8, # 20
          "model_name": r'microsoft/mdeberta-v3-base',  # r'cl-tohoku/bert-base-japanese-v2' cl-tohoku/bert-base-japanese-whole-word-masking
          "train_batch_size": 32,
          "valid_batch_size": 64,
          "max_length": 100,
          "learning_rate": 2e-5,
          "scheduler": 'cosine',
          "num_cycles": 0.5,
          "num_warmup_steps": 0,
          "min_lr": 1e-7,
          "T_max": 500,
          "weight_decay": 0.01,
          "max_grad_norm" :1.0,
          "n_accumulate": 1,
          "num_classes": 2,
          "n_fold": 4,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": HASH_NAME,
          "dropout" : 0.2,
          "debug" : True
          }

if CONFIG['model_name'] == "rinna/japanese-roberta-base":
    CONFIG["tokenizer"] = T5Tokenizer.from_pretrained(CONFIG['model_name'])
    CONFIG["tokenizer"].do_lower_case = True
    print("Vocab size: ", CONFIG["tokenizer"].vocab_size)
elif CONFIG['model_name'] == "cl-tohoku/roberta-base-japanese":
    CONFIG["tokenizer"] = RobertaJapaneseTokenizer.from_pretrained(CONFIG['model_name'])
else:
    CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
CONFIG['group'] = f'{HASH_NAME}-Baseline'


# In[10]:


def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(seed=42)


# In[11]:


df = pd.read_csv('/content/drive/MyDrive/input/nishika/hate_speech/train.csv')
print(df.head())

if CONFIG['debug']:
    df = df.head(50) 

# In[12]:


df["label"].value_counts()


# # <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Folds</h1></span>

# In[13]:


skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df["label"])):
    df.loc[val_ , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)
print(df.head())


# In[14]:


class FakenewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        self.target = df['label'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
    
        text = self.text[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
    
        target = self.target[index]
        
        onehot_t = np.zeros(CONFIG["num_classes"], dtype=np.float32) + 0.0025 ### Label smoothing
        onehot_t[target] = 0.995
        
        ids = inputs_text['input_ids']
        mask = inputs_text['attention_mask']
        
        # print(len(ids))
        # print(ids)
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'target': torch.tensor(onehot_t, dtype=torch.float)
        }

# train_dataset = FakenewsDataset(df, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
# print(train_dataset[0])

# In[15]:


class FakenewsModel(nn.Module):
    def __init__(self, cfg, config_path=None):
        super(FakenewsModel, self).__init__()
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg["model_name"], output_hidden_states=False)
        else:
            self.config = torch.load(config_path)
            
        self.model = AutoModel.from_pretrained(cfg["model_name"], config=self.config)
        self.drop = nn.Dropout(p=cfg["dropout"])
        self.fc = nn.Linear(self.config.hidden_size, cfg['num_classes'])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out.last_hidden_state[:, 0, :])
        outputs = self.fc(out)
        outputs = self.sigmoid(outputs)
        
        return outputs.squeeze()


# In[16]:


def criterion(outputs, targets):
    loss_f = nn.BCELoss()
    """
    print("output",outputs.shape, 'targets',targets.shape)
    print(outputs)
    print(targets)
    """
    return loss_f(outputs,targets)


# In[17]:


def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = ids.size(0)

        outputs = model(ids, mask)
        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=scheduler.get_lr()[0])
    gc.collect()
    
    return epoch_loss


# In[18]:


@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    predictions = []
    labels = []
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = ids.size(0)

        outputs = model(ids, mask)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
        
        preds = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        predictions.append(preds)
        labels.append(np.argmax(targets.cpu().numpy(), axis=1))
    
    gc.collect()
    
    predictions = np.concatenate(predictions, 0)
    labels = np.concatenate(labels, 0)
    
    return epoch_loss, predictions, labels


# In[42]:


def run_training(model, optimizer, scheduler, device, num_epochs, fold, savepath):
    # To automatically log gradients
    # wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    best_f1 = 0.0
    best_acc = 0.0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, predictions, labels = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        # print(predictions.shape)
        # print(labels.shape)
        f1 = f1_score(labels, predictions)
        print(f"F1-Score: {f1}")
        accuracy = accuracy_score(labels, predictions)
        print(f"f1: {f1}")
    
        # Log the metrics
#         wandb.log({"Train Loss": train_epoch_loss})
#         wandb.log({"Valid Loss": val_epoch_loss})
        
        # deep copy the model
        if best_f1 <= f1:
            print(f"{b_}Validation Acc Improved ({best_acc} ---> {f1})")
            best_f1 = f1
            # run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            #PATH = f"Loss-Fold-{fold}.bin"    ####ここのPATHは変えよう
            PATH = os.path.join(savepath , f"model-fold{fold}.bin")
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Acc: {:.4f}".format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history


# In[43]:


def prepare_loaders(f_train, df_valid):
    train_dataset = FakenewsDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = FakenewsDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader


# In[44]:


def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler


def get_scheduler(cfg, optimizer, num_train_steps):
  cfg_scheduler = cfg["scheduler"]
  num_warmup_steps = cfg["num_warmup_steps"]
  if cfg_scheduler == 'linear':
    scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps
    )
  elif cfg_scheduler == 'cosine':
      scheduler = get_cosine_schedule_with_warmup(
        optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg["num_cycles"]
      )
  return scheduler


# In[45]:


modelpath = f"models/bert-{current_time}"  
if not os.path.isdir(modelpath):
    os.makedirs(modelpath)

for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
#     run = wandb.init(project='Fakenews', 
#                      config=CONFIG,
#                      job_type='Train',
#                      group=CONFIG['group'],
#                      tags=['bert', f'{HASH_NAME}', 'bceloss'],
#                      name=f'{HASH_NAME}-fold-{fold}',
#                      anonymous='must')
    
    # Create Dataloaders
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_loader, valid_loader = prepare_loaders(df_train, df_valid)
    
    model = FakenewsModel(CONFIG)
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    # scheduler = fetch_scheduler(optimizer)
    num_train_steps = int(len(df_train) / CONFIG["train_batch_size"] * CONFIG['epochs'])
    scheduler = get_scheduler(CONFIG, optimizer, num_train_steps)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold,
                                  savepath=modelpath)
    
    # run.finish()
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()


# ## Validation

# In[47]:


# get_ipython().system('ls $modelpath')


# In[50]:


from glob import glob
MODEL_PATHS = glob(f'{modelpath}/*.bin')
MODEL_PATHS.sort()


# In[51]:


print(MODEL_PATHS)


# In[52]:


@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        
        #PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
        PREDS.append(outputs.cpu().detach().numpy()) 

    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS


# In[53]:


def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FakenewsModel(CONFIG)
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds


# In[54]:


def get_labels_frompred(out, multi=False):
    print(out)
    o = (-out).argsort(1) 
    preds = int(o[0][0].cpu().numpy())
    print(preds)

    return preds


# In[55]:


from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

fold_f1scores = []
fold_accuracyscores = []

oof_df = pd.DataFrame()
predictions = []
targets = []
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    
    train = df[df.kfold != fold].reset_index(drop=True)
    valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_loader, valid_loader = prepare_loaders(train, valid)

    valid = df[df["kfold"]==fold]

    out = inference([MODEL_PATHS[fold]], valid_loader, CONFIG['device'])
    
    preds = np.argmax(out, axis=1)
    valid["preds"] = preds
    predictions.append(preds)
    targets.append(valid["label"].values)
    
    fold_f1scores.append(f1_score(valid["label"].values, valid["preds"].values))
    fold_accuracyscores.append(accuracy_score(valid["label"].values, valid["preds"].values))
    
    # print(out.shape)
    valid["preds"] = out[:, 0]
    valid["preds2"] = out[:, 1]
    oof_df = pd.concat([oof_df, valid])
    
oof_df = oof_df.reset_index(drop=True)
oof_df.to_csv('/content/drive/MyDrive/output/nishika/hate_speech/oof_df.csv', index=False)

# In[56]:

predictions = np.concatenate(predictions, 0)
targets = np.concatenate(targets, 0)

print("Val F1-Score: ", fold_f1scores)
print("CV F1-Score: ", f1_score(targets, predictions))


# In[57]:


print("Val f1: ", fold_f1scores)
print("CV f1: ", f1_score(targets, predictions))

# In[ ]:






Fakenews-2022-11-13-16-28


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


          id       source                                               text  \
0  80074aa43     news4vip                  まともに相手されてない人との関係なんて\nそんな大事にするものか？   
1  6378fea6b  livejupiter             最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!   
2  c535f5613  livejupiter    日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな   
3  e76638295  livejupiter   よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで   
4  51e4036bf     newsplus  押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...   

   label  
0      0  
1      0  
2      1  
3      0  
4      0  
          id       source                                               text  \
0  80074aa43     news4vip                  まともに相手されてない人との関係なんて\nそんな大事にするものか？   
1  6378fea6b  livejupiter             最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!   
2  c535f5613  livejupiter    日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな   
3  e76638295  livejupiter   よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで   
4  51e4036bf     newsplus  押井は原作レイプの専門家だから\n原作マンガの真意を

Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: A100-SXM4-40GB



...

====== Fold: 3 ======


Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 1/1 [00:00<00:00,  6.02it/s]


Val F1-Score:  [0.0, 0.0, 0.0, 1.0]
CV F1-Score:  0.5
Val f1:  [0.0, 0.0, 0.0, 1.0]
CV f1:  0.5
